In [1]:
# BiLSTM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in 2021, 2022, and 2023

data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2021 and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the BiLSTM model
    model_BiLSTM = Sequential()
    model_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(seq_length, X_train.shape[2])))
    model_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_BiLSTM.add(Dropout(rate=0.5))
    model_BiLSTM.add(Flatten())
    model_BiLSTM.add(Dense(512, activation="relu"))
    model_BiLSTM.add(Dropout(rate=0.5))
    model_BiLSTM.add(Dense(64, activation="relu"))
    model_BiLSTM.add(Dense(1, activation='relu'))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiLSTM.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_BiLSTM = model_BiLSTM.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_BiLSTM = model_BiLSTM.predict(X_2023_seq)

    # Inverse scale the predictions
    y_pred_BiLSTM = scaler_y.inverse_transform(y_pred_scaled_BiLSTM)

    # Actual 2023 p_era values
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_BiLSTM = np.sqrt(mean_squared_error(y_test_actual, y_pred_BiLSTM))
    rmse_list.append(rmse_BiLSTM)

    # Calculate MAE
    mae_BiLSTM = mean_absolute_error(y_test_actual, y_pred_BiLSTM)
    mae_list.append(mae_BiLSTM)
    
    # Calculate MAPE
    mape_BiLSTM = mean_absolute_percentage_error(y_test_actual, y_pred_BiLSTM)
    mape_list.append(mape_BiLSTM)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_BiLSTM}, MAE: {mae_BiLSTM}, MAPE: {mape_BiLSTM}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')


7/7 [==============================] - 6s 6ms/step
Iteration 1/5 - RMSE: 0.616498851837409, MAE: 0.4806444924349946, MAPE: 0.13482803486103287
7/7 [==============================] - 9s 7ms/step
Iteration 2/5 - RMSE: 0.6132470271292584, MAE: 0.48532940735563557, MAPE: 0.13903870854662756
7/7 [==============================] - 6s 5ms/step
Iteration 3/5 - RMSE: 0.6137534451674593, MAE: 0.47987289857173304, MAPE: 0.14013878256302031
7/7 [==============================] - 6s 6ms/step
Iteration 4/5 - RMSE: 0.633914739949782, MAE: 0.49426374009265983, MAPE: 0.1385216515975955
7/7 [==============================] - 8s 7ms/step
Iteration 5/5 - RMSE: 0.629151993287055, MAE: 0.4936807229553443, MAPE: 0.14047018021033975
Average RMSE: 0.6213132114741927
Average MAE: 0.4867582522820735
Average MAPE: 0.13859947155572322


In [2]:
# CNN-BiLSTM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in 2021, 2022, and 2023
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2021 and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the CNN-BiLSTM model
    model_CNN_BiLSTM = Sequential()
    model_CNN_BiLSTM.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(seq_length, X_train.shape[2])))
    model_CNN_BiLSTM.add(Flatten())
    model_CNN_BiLSTM.add(Dense(64, activation='relu'))
    model_CNN_BiLSTM.add(Reshape((1, 64)))
    model_CNN_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_CNN_BiLSTM.add(Bidirectional(LSTM(64)))
    model_CNN_BiLSTM.add(Dense(1))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_CNN_BiLSTM.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_CNN_BiLSTM = model_CNN_BiLSTM.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_CNN_BiLSTM = model_CNN_BiLSTM.predict(X_2023_seq)

    # Inverse scale the predictions
    y_pred_CNN_BiLSTM = scaler_y.inverse_transform(y_pred_scaled_CNN_BiLSTM)

    # Actual 2023 p_era values
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_CNN_BiLSTM = np.sqrt(mean_squared_error(y_test_actual, y_pred_CNN_BiLSTM))
    rmse_list.append(rmse_CNN_BiLSTM)

    # Calculate MAE
    mae_CNN_BiLSTM = mean_absolute_error(y_test_actual, y_pred_CNN_BiLSTM)
    mae_list.append(mae_CNN_BiLSTM)
    
    # Calculate MAPE
    mape_CNN_BiLSTM = mean_absolute_percentage_error(y_test_actual, y_pred_CNN_BiLSTM)
    mape_list.append(mape_CNN_BiLSTM)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_CNN_BiLSTM}, MAE: {mae_CNN_BiLSTM}, MAPE: {mape_CNN_BiLSTM}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')


7/7 [==============================] - 6s 4ms/step
Iteration 1/5 - RMSE: 0.693318883479362, MAE: 0.5399108239418067, MAPE: 0.15961730549943912
7/7 [==============================] - 6s 4ms/step
Iteration 2/5 - RMSE: 0.6696432426520387, MAE: 0.5305146625537227, MAPE: 0.1492236681238492
7/7 [==============================] - 6s 4ms/step
Iteration 3/5 - RMSE: 0.6895245364224357, MAE: 0.5553212496623902, MAPE: 0.15664615118546282
7/7 [==============================] - 7s 4ms/step
Iteration 4/5 - RMSE: 0.7248804748944301, MAE: 0.5823177836597829, MAPE: 0.164429320148217
7/7 [==============================] - 7s 5ms/step
Iteration 5/5 - RMSE: 0.7106833030146279, MAE: 0.5517831636281405, MAPE: 0.1556297395951569
Average RMSE: 0.6976100880925789
Average MAE: 0.5519695366891686
Average MAPE: 0.15710923691042503


In [3]:
# BiLSTM-ED

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in 2021, 2022, and 2023
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2021 and 2022
final = common_data[common_data['year'].isin([2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 2  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the BiLSTM-ED model
    model_BiLSTM_ED = Sequential()
    model_BiLSTM_ED.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(seq_length, X_train.shape[2])))
    model_BiLSTM_ED.add(RepeatVector(seq_length))
    model_BiLSTM_ED.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_BiLSTM_ED.add(TimeDistributed(Dense(1)))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiLSTM_ED.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_BiLSTM_ED = model_BiLSTM_ED.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_BiLSTM_ED = model_BiLSTM_ED.predict(X_2023_seq)

    # Inverse scale the predictions
    y_pred_BiLSTM_ED = scaler_y.inverse_transform(y_pred_scaled_BiLSTM_ED[:, -1, :])  # Take the last time step

    # Actual 2023 p_era values
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_BiLSTM_ED = np.sqrt(mean_squared_error(y_test_actual, y_pred_BiLSTM_ED))
    rmse_list.append(rmse_BiLSTM_ED)

    # Calculate MAE
    mae_BiLSTM_ED = mean_absolute_error(y_test_actual, y_pred_BiLSTM_ED)
    mae_list.append(mae_BiLSTM_ED)
    
    # Calculate MAPE
    mape_BiLSTM_ED = mean_absolute_percentage_error(y_test_actual, y_pred_BiLSTM_ED)
    mape_list.append(mape_BiLSTM_ED)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_BiLSTM_ED}, MAE: {mae_BiLSTM_ED}, MAPE: {mape_BiLSTM_ED}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')


7/7 [==============================] - 8s 4ms/step
Iteration 1/5 - RMSE: 0.6063278467113851, MAE: 0.4778756792418622, MAPE: 0.1365150068040719
7/7 [==============================] - 7s 4ms/step
Iteration 2/5 - RMSE: 0.6001022927840611, MAE: 0.46722088290873354, MAPE: 0.1321012990985606
7/7 [==============================] - 12s 5ms/step
Iteration 3/5 - RMSE: 0.6189624630620023, MAE: 0.49105367708897235, MAPE: 0.13962867262990877
7/7 [==============================] - 6s 5ms/step
Iteration 4/5 - RMSE: 0.6021708380188392, MAE: 0.4741606643004118, MAPE: 0.13706753967733537
7/7 [==============================] - 4s 4ms/step
Iteration 5/5 - RMSE: 0.6176832042180801, MAE: 0.49263852130963603, MAPE: 0.14054595363680564
Average RMSE: 0.6090493289588735
Average MAE: 0.4805898849699233
Average MAPE: 0.13717169436933646
